# Midterm

In [1]:
import numpy as np
import scipy as sci
import sympy as sym
import pandas as pd
import seaborn as sns
import matplotlib as mp
import matplotlib.pyplot as plt

from numpy import exp, pi
from scipy.integrate import quad
from scipy.optimize import curve_fit 
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

In [2]:
WC = pd.read_csv('/Users/kev/Documents/Python/MATH-377---Math-Modeling/Data Sets/whooping_crane.csv', 
                 header=1)

## Question 1: Plant population

### a) Discrete time difference equation

Let: 

> $x$ be the amount of plants

> $a$ be a positive constant

> $r$ be the amount of seeds produced per plant

> $s$ be the total amount of seeds produced

We know: 

> The fraction of surviving seeds is based off: 

$$1-e^{-a/s}$$


The amount of plants next season is based off the amount of surviving seeds this season. The amount of seeds produced this season: 

$$s_n = x_n r$$

The total amount of plants next season: 

$$x_{n+1} = x_n r (1-e^{-a/(x_n r)})$$

### b) Equilibrium

To find whether the population size will remain a constant and its behaviour when perturbed from the constant we have to find the equilibrium solutions: 

Equilibrium: $x_{n+1} = x_n = x$

$$x = x r (1-e^{-a/(x_n r)})$$

$$r^{-1}  = 1-e^{-a/(x_n r)}$$ 

$$\frac{a}{xr} = -\ln (1-r^{-1})$$

$$x = - \frac{a}{r}\left[ \ln (1-r^{-1}) \right]^{-1}$$

Thus, we have two equilibrium solutions: 

> 1) $x = 0$

> 2) $x = - \frac{a}{r}\left[ \ln (1-r^{-1}) \right]^{-1}$

To evaluate the stability, we have to find: $f'(x_n) = \frac{\partial}{\partial x_n} x_{n+1}\bigg\rvert_x$

\begin{align} 
f'(x_n) &= \frac{\partial}{\partial x_n} x_n r (1-e^{-a/(x_n r)}) \bigg\rvert_x \\
        &= r - \left[r e^{-\frac{a}{x_n r}} + x_n r e^{-\frac{a}{x_n r}} \frac{a}{x_n^2 r} \right] \bigg\rvert_x \\
        & = r \left[ 1 - e^{-\frac{a}{x r}} \left(1 + \frac{a}{x r} \right) \right]
\end{align}

Evaluate stability for $x = 0$:

$$f'(x=0) = \lim_{x \rightarrow 0^+} \left[ 1 - e^{-\frac{a}{x r}} \left(1 + \frac{a}{x r} \right) \right] = \infty$$

> Therefore, this is always unstable. The plant population will go to infinity. (However, would it be correct to say that as the since the population is zero, there is no way for the plants to reproduce? Realistically, the population will always be stable at zero. This is where the model breaks down.)

> There is no constant to approach; therefore, any perturbation will have no affect on the behaviour of the population. 

Evaluate stability for $x = - \frac{a}{r}\left[ \ln (1-r^{-1}) \right]^{-1}$: 

\begin{align}
f'\left(x = - \frac{a}{r}\left[ \ln (1-r^{-1}) \right]^{-1} \right) &= r \left[ 1 - e^{-\frac{a}{- \frac{a}{r}\left[ \ln (1-r^{-1}) \right]^{-1} r}} \left(1 + \frac{a}{- \frac{a}{r}\left[ \ln (1-r^{-1}) \right]^{-1} r} \right) \right] \\
&= r \left[1 - e^{\ln(1-r^{-1})} \left(1 + \ln(1-r^{-1}) \right) \right] \\
&= r \left[1 - (1-r^{-1}) \left(1+\ln(1-r^{-1})\right) \right]
\end{align}

> For stability: $r \left[1 - (1-r^{-1}) \left(1+\ln(1-r^{-1})\right) \right] < 1$

$$1 - (1-r^{-1}) \left(1 + \ln(1-r^{-1}) \right) < r^{-1}$$
    
$$1 - r^{-1} < (1-r^{-1}) \left(1+\ln(1-r^{-1})\right)$$
    
$$1 < 1+\ln(1-r^{-1})$$

$$0 < \ln(1-r^{-1})$$

$$1 < 1-r^{-1}$$
    
$$0 < -r^{-1}$$

$$0 > r^{-1}$$

$$\therefore r > 0$$

> As long as the amount of seeds produced per plant is greater than zero then the population size will approach $x = - \frac{a}{r}\left[ \ln (1-r^{-1}) \right]^{-1}$. Since plants don't produce a fraction of a seed, this means as long as each plant produces at least one seed, the population will approach a constant. This makes sense as plants must produce enough seeds to replace themselves for the population to have any possibility of being constant.  

> As this is a stable equilibrium, any perturbation will not cause the plant population to deviate from approaching $x = - \frac{a}{r}\left[ \ln (1-r^{-1}) \right]^{-1}$. 

### b) Conditions for survival



### c) Population simulation

In [3]:
# Probabilty 
def prob(P, a, r):
    s = P*r
    p = 1 - exp(-a/s)
    return p

# Population (simulated)
def Plant_pop(P0, a, n, r):
    Pop = np.zeros(n) # Population over time
    Pop[0] = P0 # Initial population
    for i in range(0, n-1):
        S = np.zeros(int(r*Pop[i])) # Largest possible amount of seeds produced
        for j in range(0, len(S)):
            p = prob(Pop[i], a, r) # Probability of seeds surviving
            S[j] = np.random.choice([0, 1], p=[1-p, p]) # Which seeds survive
            j += 1
        Seed = np.sum(S) # Total surviving seeds
        Pop[i+1] = Seed # Plant population next season 
        i += 1
    return Pop

# Popluation (theoretical)
def Plant_popT(P0, a, n, r):
    P =  np.zeros(n+1)
    P[0] += P0
    for i in range(0, n):
        s = P[i] * r
        P[i+1] += r * (1-exp(-a/s)) * P[i]
        i += 1
    return P

In [4]:
# Simulate it! 
# ============

sims = 3
Plant_popt = Plant_popT(P0=1, a=100, n=50, r=2)

fig, ax = plt.subplots(1, 1, figsize=(10, 4))

for i in range(0, sims):
    Plant_popA = Plant_pop(P0=1, a=100, n=50, r=2)
    ax.plot(Plant_popA, label="Simulation {0}".format(i))
    i += 1

ax.plot(Plant_popt, "--", label="Theoretical plant population")
ax.set_xlabel(r"Generation ($n$)")
ax.set_ylabel(r"Plant population ($x$)")
ax.set_title(r"Plant population per generation for {0} simulations".format(sims+1))
plt.legend()
plt.grid()
plt.show()

<Figure size 720x288 with 1 Axes>

## Question 2: Whooping crane population

Plot the crane population

In [5]:
# A graph of the current data
# ===========================

fig, ax = plt.subplots(1, 1, figsize=(10, 4))

ax.plot(WC["Year"], WC["Cranes"])
ax.scatter(WC["Year"], WC["Cranes"])
ax.set_ylabel("Population")
ax.set_xlabel("Year")
ax.set_title("Whopping crane population from 1940 to 2015")

plt.grid()
plt.show()

<Figure size 720x288 with 1 Axes>

According to the site, Birds of North America$^{[1]}$, here's what we know about whopping cranes: 

> Monogamous and mate for life, won't find other mate unless mate dies

> Lays 1 to 3 eggs annually, though seldom fledge more than 1 young

> Starts producing eggs at 3 to 4 years old 

> Average lifespan of wild whopping crane is 25 years, although they can reach over 40 years old in captivity 

Before we model the whopping crane population, we assume:
> The population is independent of area (ie. It's the total whooping crane population in the world)

> There is no resource restrictions (ie. Unlimited food and stuff to build nests)

> The starting age distribution of the whopping crane is a Maxwell-Boltzmann distribution with an average of 25 years

>> This probablity distribution was choosen because of the low birth rate. I assumed that the low birth rate of the crane will cause it's age distribution to to be similar to the age distribution of countries with low birth rates, which resembles a Maxwell-Boltzman distribution.

>> Note that I will apply this only to the initial population, as births and deaths of cranes will cause the age distribution to deviate from this distribution.

> The observed/recorded crane population is the adult crane population. Does not include chicks (cranes less than 3.5 years old) 

### Using the Maxwell-Boltzmann distribution to find the percentage of reproductive pairs

The Maxwell-Boltzmann distribution:

$$M(x) = \sqrt{\frac{2}{\pi}}  \frac{x^2}{a^3} e^{-\frac{x^2}{2a^3}}$$

> Average: 

$$\mu = 2 a \sqrt{\frac{2}{\pi}}$$

> Setting our average to be 25 ($\mu = 25$):
    
$$a = \left( \frac{25}{2}\right) \sqrt{\frac{\pi}{2}} \approx 15.666$$

> As the cranes become fertile at around 3 to 4 years, we take the intergal of the population from 3.5 to $\infty$ to find the percentage of fertile pairs

In [6]:
def a(mu):
    a = (mu/2) * (pi/2)**0.5
    return a

def MB(x, mu):
    A = a(mu)
    M = (2/pi)**0.5 * (x**2/A**3) * exp(-x**2 / (2*A**2))
    return M

In [7]:
Ratio = quad(lambda x: MB(x, 25), 3.5, np.inf)[0]/quad(lambda x: MB(x, 25), 0, np.inf)[0]
print("Fertile ratio = {0:.4f}".format(Ratio))

Fertile ratio = 0.9971


As the fertile ratio is very high and our initial starting population is 22 cranes, let's assume all those 22 cranes are adult cranes and capable of reproduction.

### Setting up the model

#### Adults:
The adult whopping crane popultion ($x$) is determined by the diffrence relation:
$$x_{n+1} = [1 + B(x_{n-3}) - D(x_n)] x_n$$

$$\text{Where} \left\{ \begin{array}{ll}
            B(x_n) & \text{is a function determining births based on current population} \\
            D(x_n) & \text{is a function determining deaths based on current population}
            \end{array} \right.$$    
            
Note: We choose $x_{n-3}$ for the birth function as chicks take 3.5 years to mature into adults

**Birth function $B(x_n)$:**

> The birth function is primarly determined by the fertile pairs and the amount of eggs each pair produces. 

> Let: 
>> $E$ be the amount of eggs each pair produces. As the eggs are between 1 and 3, let $E=2$ for the average eggs produced per pair.

>> $F$ be the ratio of fertile pairs.

$$\therefore B(x_n) \approx x_n E$$ 


**Death function $D(x_n)$:**

> The death function includes natural death, death by predation, and death of chicks. 

> Let: 

>> $N$ be coefficient determining death due to natural causes (old age, accidents) 

>> $P$ be the coefficient determining death due to predation

>> $A$ and $B$ are constants

$$\therefore D(x_n) = N x_n + Ae^{Px_n^B}$$

>> We assumed death due to predation is an exponential function, because as population increases there is a higher chance of a predator finding the crane. Also, the increasd food supply for the predator also increases predator population.

#### Chicks
The chick ($y$) population is determined by the birth function ($B(x_n)$). Here we set up an array that groups chick based on age. Then apply the death function (for chicks). We assume only a percentage ($C$) of the chicks survive each year.

$$c = [B(x_{n}), \ C B(x_{n-1}), \ C^2 B(x_{n-2})]$$

The total population of the chicks are just the sum of the elements in the array. The reason for introducing a separate death function for the chicks is that predators act differently depending on the maturity of the crane. Some predators will only hunt adults, while others may only go for chicks. 

#### Difference equation for total population: 

The total population: 

$$P(x_{n}) = x_n + \sum_{i=1} c_i - D(x_n)$$

However, we only care about the population for adult cranes, because that is what we observe. The difference equation for adult crane population: 

$$x_{n+1} = x_n + C^2 B(x_{n-2})- D(x_n)$$

Here we have included the last entry in array "c" as the chicks have matured into adults.

### Numerical approach: 

We are going to create three arrays: 

1. Surviving infertile chicks
2. Fertile cranes
3. Total crane population

We are going to subject these arrays to the death, in addition to birth for the fertile cranes, according to the above defined functions 

In [8]:
# Difference equation for model
# =============================

def pop(n, P0, F, E, C, N, P, A, B):
    POP = np.zeros(int(n)) # Array of the total crane population over time
    POP[0] = P0 # The first entry of the total population is the initial condition
    CRANE = np.zeros(int(n)) # Array of the adult crane population
    CRANE[0] = int(P0*E/F) # Enter initial condition for adult crane population 
    CHICKS = np.zeros(3) # Array of chicks population according to years old 
    for i in range(0, n-1):
        Birth = int(CRANE[i]*E) # Birth equation
        Death = int(N*CRANE[i] + A*exp(P * CRANE[i]**B)) # Death equation (adults only)
        CHICKS[0] = Birth # Enter the first entry as the new births 
#         print(Death, Birth, CHICKS, CRANE[i], POP[i]) # For debugging, please ignore
        POP[i+1] = int(np.sum(CHICKS) + CRANE[i] - Death) # Generate total population for next year 
        CRANE[i+1] = (CRANE[i] + CHICKS[-1] - Death) # Adult crane population for next year 
        CHICKS = (CHICKS * C).astype(int) # Apply death to chicks (assuming only a ratio of C survives each year)
        CHICKS = np.roll(CHICKS, 1) # Shift the chick array forward to simulate chicks aging 
        i += 1
    return POP, CRANE

Note: The function records the populations as integers, as cranes only comes in integer quantities. 

Normally, I would plug the function into a function fit algorithm to solve for the coefficients, but due to the array manipulation and the operations there is no curve fit algorithm that is capable of solving for these coefficients. I do not have the skills nor time to code an advanced curve fitting algorithm, so we would have to make smart assumptions and guess for the coefficents, then use trail and error to come up with a function that is best representative of the data.

> $F\approx 0.9971$: This is what we have calculated from the Maxwell-Boltzmann distribution 

> $E = 1$: We set this to one as each pair produces 1 to 3 eggs. We take the median amount of eggs, which gives us 1 egg per pair of cranes. 

> $C=0.34$: The lowest chick survival percentage acceptable for crane population growth, as cranes seldom fledge more than one young, and we also factor predation and chicks dying through natural causes. (It's a tough being a chick.)

> $N=0.05$: We assume 5 % of adult cranes die each year from natural causes.

> $P, A, B$: We primarly use trail and error to guess these values such that our fit best fit the data. We also use trial and error to fine tune the above coefficients as well. 

In [14]:
# Generate the prediction based on our model
# ==========================================

# Test1 = pop(P0=WC["Cranes"].iloc[0], F=Ratio, E=1, C=0.34, N=0.01, P=0.803, A=0.2005, B=0.3, n=80)
A_pop = pop(P0=WC["Cranes"].iloc[0], F=Ratio, E=1, C=0.34, N=0.05, P=0.5, A=0.1, B=0.365, n=81)

In [15]:
# Plot it out
# ===========

fig, ax2 = plt.subplots(1, 1, figsize=(10, 4))
ax2.scatter(WC["Year"], WC["Cranes"], s=5, zorder=3)
ax2.plot(np.arange(WC["Year"].iloc[0], 2021, 1), A_pop[1], c='orange', zorder=0, label="Model predicion")
ax2.set_ylabel("Population")
ax2.set_xlabel("Year")
ax2.set_title("Adult whopping crane population from 1940 to 2015")

plt.legend()
plt.grid()
plt.show()

<Figure size 720x288 with 1 Axes>

In [11]:
print("The predicted adult whopping crane population in 2020 is approximately {0:.0f} cranes.".format(A_pop[1][-1]))

The predicted adult whopping crane population in 2020 is approximately 395 cranes.


### Shortcomings and limitations:

> The major issue of our model is that there is no available curve fitting algorithm that is capable of generating a curve of best fit based on our model. This limitation prevents us from generating a confidence interval for our predictions, and prevents us from gaining valuable insights that might arise from knowing the best values for our coefficients. Our coefficients can only be obtained by using assumptions and prior knowledge of the cranes. 

> We also assumed the initial observed crane population is the total crane population. In fact, this is a sample of the total crane population which differs from the total population. We also assumed that this population is composed entirely of adults, neglecting the chick population. 

> We assumed the percentage of surviving chicks are based on a percentage. We could improve this model by introducing a more sophisticated death function for the chicks. ie. An exponential predator term for the death function based on chick population, similar to that for the adult cranes. 

> We also did not factor in the possible increases in chick survivability due to the crane population. 

> Our model assumed that once the chicks reach maturity they are able to immediately find a mate and reproduce. It fails to factor in conditions that affects the reproduction ability of the cranes, such as the ability to find a mate, mating period, ability to build a nest...etc. We also assume the eggs hatich instantly, with no regards to incubation period. 

> We assumed ideal conditions for the cranes to reproduce, with the exception of predators. We did not account for the affects of resource quantity on the population growth of the cranes.

> The predation term in the death function is not based off a predator-prey model. We assumed that the cranes that gets eaten is soley based off the current population of the cranes. In fact, the predator population may be affected by the crane population which alters this term. We have neglected the interactions between predator and prey in this simulation. 


### Additional check: 

Let's check that our model approaches an equilibrium as time goes to infinity. (ie. The population does not blow up to infinity as time goes to infinity.) To do this, we make a plot of our model for a very long time:

In [32]:
# Generate the prediction based on our model
# ==========================================

N= 251
period = WC["Year"].iloc[0] + N

A_pop_ck = pop(P0=WC["Cranes"].iloc[0], F=Ratio, E=1, C=0.34, N=0.05, P=0.5, A=0.1, B=0.365, n=N)

# Plot it out
# ===========

fig, ax3 = plt.subplots(1, 1, figsize=(10, 4))
ax3.scatter(WC["Year"], WC["Cranes"], s=5, zorder=3)
ax3.plot(np.arange(WC["Year"].iloc[0], period, 1), A_pop_ck[1], c='orange', zorder=0, label="Model predicion")
ax3.plot([WC["Year"].iloc[0], period], [A_pop_ck[1][-1], A_pop_ck[1][-1]], '--', c='g', zorder=1)
ax3.set_ylabel("Population")
ax3.set_xlabel("Year")
ax3.set_title("Adult whopping crane population from 1940 to 2015")

plt.legend()
plt.grid()
plt.show()

# Print equilibrium population
# ============================
print("Equilibrium population: {0:.0f} adult whopping cranes".format(A_pop_ck[1][-1]))

<Figure size 720x288 with 1 Axes>

Equilibrium population: 1209 adult whopping cranes


It appears that our model does not blow up to infinity or have unreasonable behaviour at high time values, hence, I can conclude this is a suitable model. Whether this is a reasonable conclusion is up to you (the marker) to decide, and I hope you agree as well. 

## Referances: 
> [1] "Whooping Crane - Introduction | Birds of North America Online", Birdsna.org, 2019. [Online]. Available: https://birdsna.org/Species-Account/bna/species/whocra/introduction. [Accessed: 28- Feb- 2019].